In [8]:
import math
import requests
from PIL import Image
from io import BytesIO

In [51]:
def calculate_box(lat_center, lon_center, distance_km):
    # Convert distance to degrees
    delta_lat = distance_km / 111  # 1 degree of latitude is approximately 111 km
    delta_lon = distance_km / (111 * math.cos(math.radians(lat_center)))

    # Calculate corner points
    top_left = (lat_center + delta_lat, lon_center - delta_lon)
    top_right = (lat_center + delta_lat, lon_center + delta_lon)
    bottom_left = (lat_center - delta_lat, lon_center - delta_lon)
    bottom_right = (lat_center - delta_lat, lon_center + delta_lon)

    return top_left, top_right, bottom_left, bottom_right

# Example usage:
lat_center, lon_center = 9.052935, 38.735117  # Addis Ababa
distance_km = 5  # Half of the side length to get 10 km by 10 km box

In [55]:
coords = calculate_box(lat_center, lon_center, distance_km)

In [64]:
import requests
from PIL import Image
from io import BytesIO

def get_static_map_image(top_left, top_right, bottom_left, bottom_right, api_key):
    # Google Static Maps API endpoint
    endpoint = "https://maps.googleapis.com/maps/api/staticmap"
    
    # Calculate center point
    lat_center = (top_left[0] + bottom_right[0]) / 2
    lon_center = (top_left[1] + bottom_right[1]) / 2
    center = f"{lat_center},{lon_center}"

    # Calculate the size of the map (note: max size is 640x640 for free tier)
    width_km = haversine(top_left[0], top_left[1], top_right[0], top_right[1])
    height_km = haversine(top_left[0], top_left[1], bottom_left[0], bottom_left[1])
    
    # Assuming 100 pixels per km for simplicity, adjust as needed
    width_px = int(width_km * 100)
    height_px = int(height_km * 100)
    size = f"{min(width_px, 640)}x{min(height_px, 640)}"
    maptype = 'satellite'
    
    # Construct the URL for the API request
    url = f"{endpoint}?center={center}&maptype={maptype}&zoom=16&size={size}&key={api_key}"

    # Send the request to the API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Open the image from the response content
        image = Image.open(BytesIO(response.content))
        image.show()  # Display the image
        image.save("static_map.png")  # Save the image locally
        print("Image saved as 'static_map.png'")
    else:
        print("Failed to retrieve the map image:", response.status_code)

def haversine(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate the distance between two points on the Earth
    R = 6371  # Earth radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Example usage:
top_left = coords[0]
top_right = coords[1]
bottom_left = coords[2]
bottom_right = coords[3]
api_key = 

get_static_map_image(top_left, top_right, bottom_left, bottom_right, api_key)


Image saved as 'static_map.png'


In [ ]:
center = lat,lon
circum_earth_km = 40_075
size=size # pixels squared e.g. 256x256 pixels
scale = scale #if we apply scale 2 rather than default = 1, hq image 
#               and size is multiplied by 2 in either direction
#               e.g. if size = 256 and scale = 2, size(pixels) = 512x512

# At zoom 0, every pixel = 156.25km (based on size of 256x256px)

# To find distance of 1px:
# len_px = 40,075(=circum_earth_km)/size * 2**n(= zoom) 
# - every zoom level increases zoom by order of magnitude 2

# For 640x640 images, with 'scale' 2, 'size' = 1280
# 'zoom' = n
# we want: 1280len_px=10km
# so: len_px = 1/128
# hence find n = 'zoom':
# 1/128 = 40,075 / (1280 * 2**n)
# 1280 / (128 * 40,075) = 2**n
# use logs to solve for n:
# n = ln(10/40,075)/ln(2)

In [60]:
math.log(10/40075)/math.log(2)

-11.96848680504191

In [62]:
import math

def calculate_zoom_level(area_km, image_pixels, scale):
    # Earth's circumference at the equator in kilometers
    earth_circumference = 40075.0

    # Effective image resolution in pixels (considering scale factor)
    effective_pixels = image_pixels * scale

    # Ground resolution in kilometers per pixel
    ground_resolution = area_km / effective_pixels

    # Calculate the zoom level
    zoom_level = math.log2(earth_circumference / (256 * ground_resolution))
    
    return zoom_level

# Given parameters
center_lat = 9.052935
center_lon = 38.735117
area_km = 10  # width and height of the area in kilometers
image_pixels = 640  # width and height of the image in pixels
scale = 2  # scale factor

zoom_level = calculate_zoom_level(area_km, image_pixels, scale)
print(f"The required zoom level is approximately: {zoom_level:.2f}")

The required zoom level is approximately: 14.29
